# **OML Lab 6**

## Q1

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

data = pd.read_excel("C:/Users/YASH MANIYA/Desktop/Python/OML Labs/2_col_revised.xlsx", header=None)

print(data.shape)

col_prices_sheet = np.array(data.iloc[:-1, 0]).astype(float).reshape(-1,1)
print(col_prices_sheet.shape)
area = np.array(data.iloc[:-1, 1]).astype(float).reshape(-1,1)

R = 3
lam = abs(R / 10 - 5)
A = area.reshape(-1, 1)  # A matrix with only area
y = col_prices_sheet  # Target vector

ATA = np.dot(A.T, A)
ATy = np.dot(A.T, y)
optimal_coefficients = np.linalg.solve(ATA + lam * np.identity(1), ATy).ravel()

print("Optimal Coefficients (x):", optimal_coefficients)

(100, 2)
(99, 1)
Optimal Coefficients (x): [0.09877581]


## Q2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
df = pd.read_csv("C:/Users/YASH MANIYA/Desktop/Python/OML Labs/6 columns.csv")
X = df[['area', 'bedrooms', 'bathrooms']]
y = df['price']
R = 3
lam = abs(R / 10 - 5)
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)

X_scaled = (X - X_mean) / X_std

X_scaled_with_bias = np.column_stack((np.ones(X_scaled.shape[0]), X_scaled))

n, p = X_scaled_with_bias.shape
identity_matrix = np.identity(p)
identity_matrix[0, 0] = 0  # Don't regularize the bias term
X_transpose = X_scaled_with_bias.T
coeff = np.linalg.inv(X_transpose.dot(X_scaled_with_bias) + lam * identity_matrix).dot(X_transpose).dot(y)

print("Coefficients (a):", coeff[1:])
print("Intercept (b):", coeff[0])

Coefficients (a): [669588.38915016 491223.45561806 814570.45853753]
Intercept (b): 4922747.9


## Q3

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def gradient_descent(X, y, coeff, alpha, lambda_, num_iterations):
    m, n = X.shape
    for _ in range(num_iterations):
        h = sigmoid(X.dot(coeff))
        gradient = (1 / m) * X.T.dot(h - y) + (lambda_ / m) * coeff
        coeff -= alpha * gradient
    return coeff
data = pd.read_csv("C:/Users/YASH MANIYA/Desktop/Python/OML Labs/6 columns.csv")
X = data[['area', 'bedrooms', 'bathrooms']]
y = data['price']

X = np.column_stack((np.ones(X.shape[0]), X))

coeff = np.zeros(X.shape[1])
alpha = 0.01
lambda_ = 0.1
num_iterations = 1000

optimized_coeff = gradient_descent(X, y, coeff, alpha, lambda_, num_iterations)
print("Optimized coeff is: ",optimized_coeff )

Optimized coeff is:  [4.89823938e+07 2.67019629e+11 1.49835630e+08 7.07404218e+07]


In [ ]:
data = pd.read_csv('diabetes2.csv')
X = data.drop('Outcome', axis=1)
y = data['Outcome']
X['intercept'] = 1

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_loss(theta, X, y, lambda_reg):
    m = len(y)
    h = sigmoid(X.dot(theta))
    regularization_term = (lambda_reg / (2 * m)) * np.sum(theta[1:]**2)
    J = (-1 / m) * (y.dot(np.log(h)) + (1 - y).dot(np.log(1 - h))) + regularization_term
    return J

def logistic_gradient(theta, X, y, lambda_reg):
    m = len(y)
    h = sigmoid(X.dot(theta))
    gradient = (1 / m) * X.T.dot(h - y)
    gradient[1:] += (lambda_reg / m) * theta[1:]
    return gradient

def logistic_hessian(theta, X, y, lambda_reg):
    m = len(y)
    h = sigmoid(X.dot(theta))
    diagonal = (1 / m) * (h * (1 - h)).sum()
    X_array = X.values  # Convert DataFrame to a numpy array
    hessian = (1 / m) * X_array.T.dot(X_array)
    np.fill_diagonal(hessian, np.diag(hessian) + (lambda_reg / m))
    return hessian

theta_initial = np.zeros(X.shape[1])
lambda_reg = 1.0
learning_rate = 0.1
tolerance = 1e-6
theta = theta_initial.copy()
for _ in range(1000):
    gradient = logistic_gradient(theta, X, y, lambda_reg)
    theta -= learning_rate * gradient
    if np.linalg.norm(gradient) < tolerance:
        break

print("Optimal Theta (Gradient Descent):", theta.tolist())

symmetric_matrix = np.random.uniform(5, 10, (X.shape[1], X.shape[1]))
for i in range(X.shape[1]):
    symmetric_matrix[i, i] = np.random.uniform(5, 10)
for i in range(X.shape[1]):
    for j in range(i + 1, X.shape[1]):
        symmetric_matrix[i, j] = symmetric_matrix[j, i] = np.random.uniform(0, 1)

L = np.max(np.linalg.eigvalsh(symmetric_matrix))  # Lipschitz constant
theta = theta_initial.copy()
for _ in range(1000):  # Maximum number of iterations
    gradient = logistic_gradient(theta, X, y, lambda_reg)
    theta = theta.dot(np.linalg.inv(np.eye(X.shape[1]) + (learning_rate / L) * symmetric_matrix))
    if np.linalg.norm(gradient) < tolerance:
        break

print("Optimal Theta (Mirror Descent):", theta)

theta = theta_initial.copy()
for _ in range(1000):  # Maximum number of iterations
    gradient = logistic_gradient(theta, X, y, lambda_reg)
    hessian = logistic_hessian(theta, X, y, lambda_reg)
    theta -= np.linalg.inv(hessian).dot(gradient)
    if np.linalg.norm(gradient) < tolerance:
        break

print("Optimal Theta (Newton's Method):", theta)


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Optimal Theta (Gradient Descent): [14.22887271662626, -1.4284340192269855, -11.058051932987281, -1.0923250731825889, -3.1377913635368966, 0.31623596069950066, 0.6861023372022733, -1.2052283672426836, -3.2633753955105695]
Optimal Theta (Mirror Descent): [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Optimal Theta (Newton's Method): [ 1.17172262e-01  2.83915216e-02 -1.68897401e-02  7.55630139e-04
 -6.42815874e-04  5.97833701e-02  6.77479820e-01  7.22644307e-03
 -5.89285545e+00]


## Q4

In [ ]:
def objective_function(x):
    return 0.5 * (x[0] - 2)**2 + 0.5 * (x[1] - 2)**2 + 0.5 * np.linalg.norm(x, ord=1)

def poximal(x, lambda_):
    return np.sign(x) * np.maximum(0, np.abs(x) - lambda_)

x = np.array([0.0, 0.0])
alpha1 = 1/2
r = 11
max_itr = 100
for iteration in range(max_itr):
    grad = np.array([x[0] - 2, x[1] - 2]) + np.sign(x)
    x = poximal(x - alpha1 * grad, alpha1)


print("Result for alpha1 (1/2):", x)

x = np.array([0.0, 0.0])

alpha2 = 1 / (r + np.arange(1, max_itr + 1))
for iteration in range(max_itr):
    grad = np.array([x[0] - 2, x[1] - 2]) + np.sign(x)
    x = poximal(x - alpha2[iteration] * grad, alpha2[iteration])

print("Result for alpha2 (1/(r + k)): ", x)


Result for alpha1 (1/2): [1.11022302e-16 1.11022302e-16]
Result for alpha2 (1/(r + k)):  [0.00900901 0.00900901]


## Q5

In [ ]:
data = pd.read_excel('C:/Users/YASH MANIYA/Desktop/Python/OML Labs/2_col_revised.xlsx', header=None)  # Replace 'house_data.xlsx' with your file path
X = data.iloc[:98,0].values  # Area (in sq feet)
y = data.iloc[:98,1].values  # House price (in Rs)

X = (X - X.mean()) / X.std()
y = (y - y.mean()) / y.std()

def cost_function(x, X, y, lambda_):
    residuals = X.dot(x) - y
    mse = 0.5 * np.mean(residuals**2)
    l1_reg = lambda_ * np.sum(np.abs(x))
    return mse + l1_reg

def proximal_operator(x, alpha):
    return np.sign(x) * np.maximum(0, np.abs(x) - alpha)

lambda_ = np.abs(11 / 10 - 5)
alpha_k = 1 / 2  # You can also use 1 / (r + k) if needed
x = np.zeros(len(X))  # Initialize x with zeros
iterations = 1000  # Choose an appropriate number of iterations

for k in range(iterations):
    gradient = X.T.dot(X.dot(x) - y) / len(y)
    x = proximal_operator(x - alpha_k * gradient, alpha_k * lambda_)

x = x * y.std() / X.std()

print("Optimal coefficients (x):", x)


Optimal coefficients (x): [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0.]
